# GEMS PedTools Example Usage for R
Below is a simple example that illustrates how to access data in the PedTools database.
### Set up an HTTP client using Python's request library
We use a `Session` object to store our API key and automatically include it in the header for each request.

Note that we have a `api_key.R` file in the Exchange-Notebooks directory. The file contains only the below line.
```
api_key <- 'SECRET'
```

In [1]:
# Load necessary libraries
library(httr)
library(jsonlite)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Set up an HTTP client

In [2]:
# Grab your API Key and assign it to api_key
source("api_key.R")

# Initialize session headers
headers <- add_headers(apikey = api_key)

# Define base URL
# Uncomment the base URL you wish to use
base <- "https://exchange-1.gems.msi.umn.edu/pedtools/v1"
# base <- "https://exchange-dev.gems.msi.umn.edu/pedtools/v1"


### Grab all information for a specific variety

In [3]:
# Query information for a specific variety
params <- list(pedigree_depth = 5)
variety <- URLencode("TURKEY", reserved = TRUE)
url <- paste0(base, "/", variety)

# Perform GET request
response <- GET(url, headers, query = params)

# Check if the request was successful
if (status_code(response) == 200) {
  # Parse JSON response and convert to a data frame
  content <- content(response, as = "text", encoding = "UTF-8")
  data <- fromJSON(content, flatten = TRUE) %>% as.data.frame()
  print(data)
} else {
  cat("Error:", status_code(response), "-", content(response, as = "text"), "\n")
}

    preferred_name crop_name is_landrace backcross_depth selfing_count
1   TURKEY GID:135     wheat       FALSE               0             2
2 TURKEY GID:10509     wheat       FALSE               0             0
  market_class release_date developer parentage
1           NA           NA        NA        NA
2           NA           NA        NA        NA
                                                                                                                                   aliases
1 135, CWI51783, P.1066-?-?, TK, TURKEY, germplasm_bank_id, germplasm_bank_accession_id, selection_history, cross_abbreviation, cross_name
2                                                                          10509, TURKEY, TURKEY 13.R, germplasm_bank_id, bcid, cross_name
  father              pedigree mother.preferred_name mother.crop_name
1     NA P.1066 GID:25 (25) F2     P.1066-? GID:3700            wheat
2     NA      TURKEY GID:10509                  <NA>             <NA>
  mother.is_land

### What are the aliases for the returned varieties?

In [4]:
if ("aliases" %in% colnames(data)) {
    print("Aliases for the varieties:")
    print(data$aliases)
} else {
    print("No aliases found.")
}

[1] "Aliases for the varieties:"
[[1]]
        name                        type
1        135           germplasm_bank_id
2   CWI51783 germplasm_bank_accession_id
3 P.1066-?-?           selection_history
4         TK          cross_abbreviation
5     TURKEY                  cross_name

[[2]]
         name              type
1       10509 germplasm_bank_id
2      TURKEY              bcid
3 TURKEY 13.R        cross_name



### Let's check another entry, and retrieve its pedigree to a depth of 5 (great-great-great grandparents).

In [5]:
# Query pedigree information for a specific variety
params <- list(pedigree_depth = 5)
variety <- URLencode("SANDPIPER", reserved = TRUE)
url <- paste0(base, "/", variety)

# Perform GET request
response <- GET(url, headers, query = params)

# Check if the request was successful
if (status_code(response) == 200) {
  # Parse JSON response and convert to a data frame
  content <- content(response, as = "text", encoding = "UTF-8")
  data <- fromJSON(content, flatten = TRUE) %>% as.data.frame()
  print(data$pedigree)
} else {
  cat("Error:", status_code(response), "-", content(response, as = "text"), "\n")
}

[1] "###123561 / STEWART 63 /2/ 980 GID:980 / WELLS /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /4/ POLONICUM PI185309 /2/ ###250171 / TEHUACAN 60 /3/ ###123633 / TEHUACAN 60 /2/ 980 GID:980 / WELLS /5/ II23584 GID:417 (417) F6 /2/ ###6385 (###6385) F2 /3/ ###3926 (###3926) F1 (II23584 GID:417 (417) F6 /2/ ###6385 (###6385) F2 /3/ ###3926 (###3926) F1) F6"


### How do I find the Coefficient of Parentage (COP) between any arbitrary pair of varieties?
We use the `/{variety1}/{variety2}/cop` endpoint to obtain the COP between two varieties, variety1 and variety2.

In [6]:
params <- list(pedigree_depth = 5)
var1 <- URLencode("###33476", reserved = TRUE)
var2 <- URLencode("MEDITERRANEAN", reserved = TRUE)
url <- paste0(base, "/", var1, "/", var2, "/cop")

# Perform GET request
response <- GET(url, headers, query = params)

# Check if the request was successful
if (status_code(response) == 200) {
  # Parse JSON response and convert to a data frame
  content <- content(response, as = "text", encoding = "UTF-8")
  data <- fromJSON(content, flatten = TRUE) %>% as.data.frame()
  print(data)
} else {
  cat("Error:", status_code(response), "-", content(response, as = "text"), "\n")
}

    cop
1 0.125
